![](https://global-uploads.webflow.com/5f4444910aa0ad6a50bb4f52/5f444fb00e4dc15dd0f0416e_sisu-logo.svg)

# Task List Processing <span style = "color:yellow">Version 10</span>

# <span style="color:red"> Dear CS Team (please read) 10/25/2022</span>

I have made a change to this process. 

I am shifting the SQL statement to gather the **agent information** to the front of the import process. My reasoning for doing this is to avoid an issue. The issue is that if the assignee is not in the Sisu User Database, then there is no assignee number for the task to be assigned to and will result in the task being uploaded without an assignee designated. 

In concert with the position change, I have built a check that will alert if there are new agents. I did not put this check later, where we were initially pulling in agent data, for convenience and also for the process. If we find out at that point, we will already have the task lists imported and having to pause and wait will burden our bandwidth (mentally). This way, if we get the error it is no big deal because we haven't started the process. (Stopping later also messes up the count within the tool -- not a huge deal, but worth avoiding.)

The check will tell you the new user name and how many tasks the user is assigned to. It will also tell you the total lists affected. 

The rest of the process will remain the same **EXCEPT** that step 5 will no longer give an SQL statement for getting the agent info (as that will already be gathered) and step 6 will no longer collect the agent data. 



-Josh

NOTES:

- if the users are not already in Sisu the matchup will not work. 
    - Need to set up an alert for when the users are not in the Sisu and initiate the user import. 

<div style = "text-align: left"> The purpose of this notebook is to process, and reformat,
                                   Task List information, <br> brought in as an excel document,
                                   down to SQL statements for inserting into <span style = "font-family:Roboto Thin; font-size:1em; color:#FF5000">sisu</span>. </div>

## 0 <span style="color:#b54dff">Library

In [1]:
#Library 

import os
from termcolor import colored
import pwd # needed if the document is stored on computer instead of Google Drive
import tkinter as tk
from tkinter import filedialog
import pandas as pd
pd.set_option('display.max_columns', None) # keeps pandas from truncating columns
import numpy as np
import clipboard
pd.options.display.max_colwidth = 1000
import warnings
warnings.filterwarnings('ignore')
from tabulate import tabulate
from datetime import datetime
import pytz
import task_list_tools as tlt


run_list = []

# Cell feedback
def cell_feedback():
    print("Cell ran at:")
    tz_east = pytz.timezone('US/Eastern') 
    datetime_eastern = datetime.now(tz_east)
    print("Eastern:", datetime_eastern.strftime("%H:%M:%S"))

    tz_central = pytz.timezone('US/Central') 
    datetime_central = datetime.now(tz_central)
    print("Central:", datetime_central.strftime("%H:%M:%S"))

    tz_pacific = pytz.timezone('US/Pacific')
    datetime_pacific = datetime.now(tz_pacific)
    print("Pacific:", datetime_pacific.strftime("%H:%M:%S"))


import subprocess
import platform

def raise_app(root: tk):
    root.attributes("-topmost", True)
    if platform.system() == 'Darwin':
        tmpl = 'tell application "System Events" to set frontmost of every process whose unix id is {} to true'
        script = tmpl.format(os.getpid())
        output = subprocess.check_call(['/usr/bin/osascript', '-e', script])
    root.after(0, lambda: root.attributes("-topmost", False))


## 1 <span style="color:#8b49fc"> Select Client Template File, View Lists & Tasks and Clean & Fixes 

<span style = "color:#73efff"> Make sure that the file you are working with is in a *FOLDER* that is formated with the team ID, a space, and then the team name. For example: 12345 Pretend Realty

In [2]:
'''
This cell collects the template file, does a few transformations, and validates the date. It will give Warnings and Errors if there is an concerne or issue with the data. 
'''

team_id, team_name, df = tlt.get_task_list_file_and_validate()

tlt.cell_feedback()

Team ID:	1177
Team Name:	The Suarez Team
File Name:	Task List for The Suarez Team (updated).xlsx
Sheet 1 (i=0):   	START HERE => Enter your Users
Sheet 2 (i=1):   	Master List (Make a copy)
Sheet 3 (i=2):   	Full Listing Launch Procedure
Sheet 4 (i=3):   	Full Listing Launch w CS Campai
Sheet 5 (i=4):   	Full Listing Launch w Renovatio
Sheet 6 (i=5):   	Sheet1
Sheet 7 (i=6):   	Data mapping (hidden)
 
 
SHEETS WITH TASKS AND COUNT OF TASKS/ROWS:
0 rows in Master List (Make a copy)
86 rows in Full Listing Launch Procedure
104 rows in Full Listing Launch w CS Campai
99 rows in Full Listing Launch w Renovatio
68 rows in Sheet1
357 rows in total.
 
All empty Day values have been set to zero.
All whitespace cleared from string values.
 
GOOD: NO duplicates found in data.
 
|    | sheet_name                    | Task Name                                                   |
|---:|:------------------------------|:------------------------------------------------------------|
| 67 | Full Listing

## 2 <span style = "color:#577eff">Collect Agent Information </span> 

In [3]:
'''
This cell saves a query to the clipboard for agent info. 
In case the data is deleted somehow, the data is saved to the sql_get_agent_info variable.
'''

sql_get_agent_info = tlt.get_agent_info(team_id)

tlt.cell_feedback()

Get Agent SQL query has been copied to your clipboard. 
Paste this into Sisu's Raw Data Tool
Copy the returned table from the Raw Data Tool and run the following cell.
 
Cell ran at:
Eastern: 17:31:48
Central: 16:31:48
Pacific: 14:31:48


## 3 <span style = "color:#5795ff">Retrieve Agent Information and Collect Current Task Lists </span> 

In [4]:
df_agents = tlt.capture_agent_info_and_check(df)

''' 
This cell produces query for retreiving the current Task Lists
The query text will automatically save to the clipboard, but in the case that the clipboard is cleared
the texted is also saved to the sql_get_task_lists variable. 
'''
sql_get_task_lists = tlt.get_task_lists(team_id)

tlt.cell_feedback()


STOP!!  This data has users that are not yet imported into SISU.
Jonathan Sanchez    34
Name: Assign to TC, Agent or assignee full name, dtype: int64
34 total tasks affected.
 
Get current Task Lists SQL has been copied to your clipboard. 
Paste this into Sisu's Raw Data Tool
Copy the returned table from the Raw Data Tool and run the following cell.
 
Cell ran at:
Eastern: 17:31:58
Central: 16:31:58
Pacific: 14:31:58


## 4 <span style = "color: #40a0bd"> Task List Insert </span>

In [5]:

'''
Retrieve copied data from SQL output
Also define a Reset DataFrame
'''

df_reset_1, current_task_list_names = tlt.retrieve_current_task_lists_data(df)

'''
Getting information about the Task List count
'''
final_task_list_count = tlt.task_list_feedback(df, current_task_list_names)

'''
Adding missing columns to the DataFrame
'''

df, team_id = tlt.adding_columns(df, team_id)

'''
Define client task lists for INSERT statement
'''

df_client_task_list, client_task_list_cols_order = tlt.define_client_task_list(df, current_task_list_names, final_task_list_count)


'''
This cell produces an INSERT statement for the new Task Lists. 
The statment will automatically be saved to the clipboard but in the case that its deleted
the statement is saved to sql_insert_task_lists.capitalize

If you need to use sql_insert_task_lists, you can just type it in a code cell and it will output,
but the easiest way to use it is to type(copy) the following into a code cell:

clipboard.copy(sql_insert_task_lists)

'''

df, df_client_task_list, sql_insert_task_lists = tlt.insert_task_lists(df, df_client_task_list, team_id)


tlt.cell_feedback()

TASK LIST COUNT
4 Current Task Lists
4 New Task Lists
8 Total Task Lists
Team ID column added.
created_ts column added.
updated_ts column added.
Status column added.
status_trigger column added.
 
Inserting Task List from the Team 1177 template
The Task List INSERT SQL from the team 1177 template has been copied to your clipboard. 
Paste into the Sisu Raw Data Tool 
This will load the Task Lists from the template into Sisu.
 
Cell ran at:
Eastern: 17:32:11
Central: 16:32:11
Pacific: 14:32:11


## 5 <span style="color:#40b9bd">Collect Current Task Blueprints </span>  

In [6]:
'''
This cell produces an SQL query for getting the current task bluprints.
The query text will automatically be saved to the clipboard but if it is deleted
the query is also saved to the sql_get_task_blueprints variable. 
'''

sql_get_task_blueprints = tlt.get_task_blueprints(team_id)

tlt.cell_feedback()

Get Tasks SQL has been copied to your clipboard. 
Paste this into Sisu's Raw Data Tool
Copy the returned table from the Raw Data Tool and run the following cell.
 
Cell ran at:
Eastern: 17:32:21
Central: 16:32:21
Pacific: 14:32:21


## 6 <span style = "color:#40bda2"> Retrieve Task Blueprints & Insert Task Blueprint </span>

In [7]:
'''
This cell retrieves the current task blueprints.
Copy the queried values in the Raw Data Tool and run this cell.
'''

current_task_blueprint = tlt.retrieve_task_blueprints()

'''
Get feedback about task counts
'''

client_task_blueprint_cols, final_task_name_count, new_task_count = tlt.task_blueprint_feedback(df, current_task_blueprint)


df, df_reset_2, df_assign_map, df_assign_map_general, df_client_task_blueprints = tlt.process_agent_info(df, df_agents, client_task_blueprint_cols)

sql_insert_task_blueprints = tlt.insert_task_blueprints(df_client_task_blueprints, team_id)

tlt.cell_feedback()

# '''
# This cell saves a query to the clipboard for agent info. 
# In case the data is deleted somehow, the data is saved to the sql_get_agent_info variable.
# '''

# sql_get_agent_info = tlt.get_agent_info(team_id)

# tlt.cell_feedback()

TASK COUNT
885 Current Tasks
357 New Tasks
1242 Total Tasks
 
Inserting Task Blueprints from the Team 1177 template
An INSERT statement for the Task Blueprints from the team 1177 template has been copied to your clipboard. 
Paste into the Sisu Raw Data Tool 
This will load the Task Blueprints from the template into Sisu.
 
Cell ran at:
Eastern: 17:32:33
Central: 16:32:33
Pacific: 14:32:33


## 7 <span style="color:#40bd72"> Collect Task List Matchup Data

In [35]:
'''
This cell creates a query for the task list matchup data. 
This will save the text for the query to the clipboard. 
If the clipboard is cleared for some reason, use the sql_task_list_matchup variable
'''

sql_task_list_matchup = tlt.get_task_list_mathcup_data(team_id)

tlt.cell_feedback()

SQL query has been copied to your clipboard. 
Paste this into Sisu's Raw Data Tool
Copy the returned table from the Raw Data Tool and run the following cell.
 
Cell ran at:
Eastern: 15:44:44
Central: 14:44:44
Pacific: 12:44:44


## 8 <span style = "color:#40bd72"> Retrieve Task Lists IDs & Collect Task Blueprint IDs </span>

In [36]:
'''
This cell retrieves the task list data for the matchup
'''

df_matchup_task_lists = tlt.retrieve_task_list_matchup_data()

'''
This cell creates a query for the task blueprint matchup data. 
This will save the text for the query to the clipboard. 
If the clipboard is cleared for some reason, use the sql_task_blueprint_matchup variable 
'''

sql_task_blueprint_matchup_data = tlt.get_task_blueprint_matchup_data(team_id)

tlt.cell_feedback()

Get Tasks SQL has been copied to your clipboard. 
Paste this into Sisu's Raw Data Tool
Copy the returned table from the Raw Data Tool and run the next cell.
 
Cell ran at:
Eastern: 15:44:59
Central: 14:44:59
Pacific: 12:44:59


## 9 <span style = "color:#40bd48"> Retrieve Task Blueprints IDs & Build the Lists-Tasks Matchup (Merge) </span>

In [37]:
'''
This cell retrieves the task blueprint matchup data.
It also sets df_reset_3
'''

df_reset_3, df_matchup_task_blueprint = tlt.retrieve_task_blueprint_matchup_data(df)

'''
Merge Data
'''

df, df_reset_3 = tlt.merge_data(df_matchup_task_blueprint, df, df_matchup_task_lists, new_task_count)

tlt.cell_feedback()

Number of Tasks before removing duplicates:  1
Number of duplications:  1.0
 
Cell ran at:
Eastern: 15:45:11
Central: 14:45:11
Pacific: 12:45:11


## 10 <span style = "color:#61bd40"> Validate Merge </span>

If not correct compare the <span style = "color:#61bd40"> 'Task Name' </span> from <span style = "color:#61bd40"> df </span> to <span style = "color:#61bd40"> 'name' </span> from <span style = "color:#61bd40">df_matchup_task_blueprint </span>

like the following: 

```python
# Look in the data frame for the rows with Null values in 'task_blueprint_id'
df[df['task_blueprint_id'].isna()]

# Then use the 'Task Name' from df as a string to be searched in df_mathup_task_blueprint 'name' column. If run correctly, this will
# give just a single row where the 'task_blueprint_id' can be found. 
df_matchup_task_blueprint[df_matchup_task_blueprint['name'].str.contains('Process closing package - 1 day')]

# finally, add the 'task_blueprint_id' to the correct row. 
df.loc[df['Task Name'].str.contains('Process closing package - 1 day'), 'task_blueprint_id'] = 269973

```

Checking against display order appears to be a good way to correct this as well. <br>
The below code has been incorporated into step 10 IF the number display orders is equal to the number of missing blueprint ids. 

```python
for i in df[df['task_blueprint_id'].isna()]['display_order']:
    print(df_matchup_task_blueprint[df_matchup_task_blueprint['display_order']==i]['task_blueprint_id'].to_list()[0])
    df.loc[df['display_order']==i, 'task_blueprint_id'] = df_matchup_task_blueprint[df_matchup_task_blueprint['display_order']==i]['task_blueprint_id'].to_list()[0]
```

In [38]:
'''
Validate Merge -- checks for Tasks that do not have task_blueprint_id
'''

tlt.validate_merge(df, df_matchup_task_blueprint)

tlt.cell_feedback()

No Null values for Task List ID
No Null values for Task Blueprint ID
 
Cell ran at:
Eastern: 15:45:14
Central: 14:45:14
Pacific: 12:45:14


## 11 <span style = "color:#93bd40"> Build the Matchup Insert Statement </span>

In [39]:
'''
Create INSERT statement for merge data.
This will automatically save to clipboard. If somehow deleted use:

clipboard.copy(sql_merge_data)
'''

sql_merge_data = tlt.create_merge_insert_statement(df, team_id)

tlt.cell_feedback()

Inserting Matchup data for Team 1846
An INSERT statement for the Matchup data for Team 1846 has been copied to your clipboard. 
Paste into the Sisu Raw Data Tool 
This will load the Matchup data into Sisu.
 
Cell ran at:
Eastern: 15:45:17
Central: 14:45:17
Pacific: 12:45:17


## 12 <span style = "color:#bdbb46"> Summary </span>

In [13]:
'''
This cell creates a summary. Copy the summary table and paste it into a markdown cell. Run the Markdown cell and copy the output. Paste the output into JIRA. 
'''

tlt.create_summary(current_task_list_names, current_task_blueprint, df)


SUMMARY
Copy the white text below and paste it into the following markdown cell. Run the markdown cell, select the output, copy it, and paste into JIRA.
| Subject                 |   Count |
|:------------------------|--------:|
| Initial Task List Count |       0 |
| New Task List Count     |       1 |
| Total Task List Count   |       1 |
| Initial Task Count      |       0 |
| New Task Count          |     151 |
| Total Task Count        |     151 |


| Subject                 |   Count |
|:------------------------|--------:|
| Initial Task List Count |       0 |
| New Task List Count     |       1 |
| Total Task List Count   |       1 |
| Initial Task Count      |       0 |
| New Task Count          |     151 |
| Total Task Count        |     151 |

## FINISH